# Meta-Kaggle ML example
Goal: predict user ranking from
 - first and last submission date
To add:
 - number of submissions made to the contest
 - number of team members
 - number of contests participated in (by leader or whole team?)
 - improve prediction: instead of raw ranking normalize by total number after dropping inactives (single submissions?), try predicting medaled or not (too imbalanced?)

In [2]:
import pandas as pd
import glob
import meta_kaggle_utils as utils

In [4]:
data_location = 'meta-kaggle/'
submission_file_name = 'Submissions.csv'
team_file_name = 'Teams.csv'

## load submissions and teams files

In [5]:
submissions = utils.load_kaggle_csv(data_location + submission_file_name)
submissions['PublicScoreFullPrecision'] = pd.to_numeric(submissions['PublicScoreFullPrecision'], errors='coerce')
submissions['PrivateScoreFullPrecision'] = pd.to_numeric(submissions['PrivateScoreFullPrecision'], errors='coerce')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (5,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


The file contains 5015235 rows.
The table contains the following columns: 
['SubmittedUserId' 'TeamId' 'SourceKernelVersionId' 'SubmissionDate'
 'ScoreDate' 'IsAfterDeadline' 'PublicScoreLeaderboardDisplay'
 'PublicScoreFullPrecision' 'PrivateScoreLeaderboardDisplay'
 'PrivateScoreFullPrecision']


In [6]:
teams = utils.load_kaggle_csv(data_location + team_file_name)
teams = teams.dropna(axis=0, how='any')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


The file contains 1740398 rows.
The table contains the following columns: 
['CompetitionId' 'TeamLeaderId' 'TeamName' 'ScoreFirstSubmittedDate'
 'LastSubmissionDate' 'PublicLeaderboardSubmissionId'
 'PrivateLeaderboardSubmissionId' 'IsBenchmark' 'Medal' 'MedalAwardDate'
 'PublicLeaderboardRank' 'PrivateLeaderboardRank']


In [7]:
## merge submission and teams files

In [8]:
# add a joining column to the submissions matrix
submissions['join_teams_submissions'] = submissions.index
submissions['PublicLeaderboardScore'] = submissions.PublicScoreFullPrecision
submissions['PrivateLeaderboardScore'] = submissions.PrivateScoreFullPrecision

(5015235, 10)
(5015235, 13)


In [9]:
# extract the column to add (and keep the joining column)
public_leaderboard = submissions[['join_teams_submissions', 'PublicLeaderboardScore']]
private_leaderboard = submissions[['join_teams_submissions', 'PrivateLeaderboardScore']]
print(public_leaderboard.shape)

(5015235, 2)


In [10]:
teams['join_teams_submissions'] = teams.PublicLeaderboardSubmissionId
print(teams.shape)

(22234, 13)


In [11]:
# perform the merge
teams_with_score = teams.join(public_leaderboard, on='join_teams_submissions', rsuffix='_j1')
teams_with_score = teams_with_score.join(private_leaderboard, on='join_teams_submissions', rsuffix='_j2')
print(teams.shape, public_leaderboard.shape, teams_with_score.shape)

# drop the columns added for joining
drop_cols = ['join_teams_submissions_j1', 'join_teams_submissions_j2', 'join_teams_submissions']
teams_with_score = teams_with_score.drop(drop_cols, axis=1)
print(teams_with_score.shape)

(22234, 13) (5015235, 2) (22234, 17)
(22234, 14)


## set up the matrix 

In [ ]:
feature_col = ''